# Выбор локации для скважины. Проект по спринту "Машинное обучение в бизнесе"

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
#импорт библиотек и инструментов
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [2]:
#загрузка данных
data1 = pd.read_csv('/datasets/geo_data_0.csv')
data2 = pd.read_csv('/datasets/geo_data_1.csv')
data3 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
#проверим корректность загрузки
display(data1.head())
display(data2.head())
display(data3.head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [4]:
# проверка данных на наличие пропусков и аномалий
display(data1.info())
display(data2.info())
display(data3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

In [5]:
#проверка на дубликаты
display((data1.duplicated()).sum())
display((data2.duplicated()).sum())
display((data3.duplicated()).sum())

0

0

0

###### Выводы
- импортировали необходимые библиотеки и инструменты
- загрузили данные
- проверили на дубликаты и пропуски
- дубликатов и пропусков нет, можно переходить к обучению модели

## Обучение и проверка модели

Необходимо создать модель для каждого региона, поэтому сделаем это, используя функцию

In [6]:
def model_prediction_rmse (data):
    #выделяем данные и целевой признак
    features = data.drop(['id', 'product'], axis=1)
    target = data['product']
    
    #делим на выборки
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=42)
    
    #стандартизация
    scaler = StandardScaler()
    features_train_scaled = scaler.fit_transform(features_train)
    features_valid_scaled = scaler.transform(features_valid)
    
    #обучение модели
    model = LinearRegression()
    model.fit(features_train_scaled, target_train)
    
    #предсказания на валидационной выборке
    predictions = model.predict(features_valid_scaled)
    
    #расчет среднего запаса и rmse
    avg_predicted_reserve = predictions.mean()
    rmse = np.sqrt(mean_squared_error(target_valid, predictions))
    
    #предсказания на всем наборе данных
    features_scaled = scaler.transform(features)
    predictions_all = model.predict(features_scaled)
    
    return avg_predicted_reserve, rmse, predictions, target_valid, predictions_all

# Применение функции для каждого региона
results_1 = model_prediction_rmse(data1)
results_2 = model_prediction_rmse(data2)
results_3 = model_prediction_rmse(data3)

print('Средний запас предсказанного сырья в регионе 1:', results_1[0])
print('RMSE модели в регионе 1:', results_1[1])

print('Средний запас предсказанного сырья в регионе 2:', results_2[0])
print('RMSE модели в регионе 2:', results_2[1])

print('Средний запас предсказанного сырья в регионе 3:', results_3[0])
print('RMSE модели в регионе 3:', results_3[1])

Средний запас предсказанного сырья в регионе 1: 92.39879990657768
RMSE модели в регионе 1: 37.75660035026169
Средний запас предсказанного сырья в регионе 2: 68.7128780391376
RMSE модели в регионе 2: 0.8902801001028846
Средний запас предсказанного сырья в регионе 3: 94.77102387765939
RMSE модели в регионе 3: 40.145872311342174


In [7]:
#добавим предсказанные значения в датасеты
data1['predictions'] = pd.Series(results_1[4])
data2['predictions'] = pd.Series(results_2[4])
data3['predictions'] = pd.Series(results_3[4])

###### Выводы
- обучили модели линейной регрессии
- сделали предсказания на валидационной выборке
- получили результаты
- средние запасы сырья и RMSE практически равны в регионе 1 и 3  
- средний запас сырья в регионе 2 сильно меньше, но при этом RMSE значительно ниже чем в регионах 1 и 3

## Подготовка к расчёту прибыли

In [8]:
#запишем необходимые значения в переменные
budget = 10000000000
revenue_barrel = 450
best = 200
research = 500
barrels_per_unit = 1000

# расчет для безубыточной разработки
cost_well = budget / best
need_vol = cost_well / (revenue_barrel * barrels_per_unit)

print('Достаточный объем сырья для безубыточной разработки:', need_vol)

Достаточный объем сырья для безубыточной разработки: 111.11111111111111


###### Вывод
Достаточный объем сырья для безубыточной разработки составил более 111 тыс. баррелей, что больше чем среднее значение в скважинах всех регионов

## Написание функции для расчёта прибыли по выбранным скважинам и предсказаниям модели

In [9]:
data_1 = (data1.iloc[:, -2:]).sort_values(by='predictions', ascending=False)
data_2 = (data2.iloc[:, -2:]).sort_values(by='predictions', ascending=False)
data_3 = (data3.iloc[:, -2:]).sort_values(by='predictions', ascending=False)
display(data_1.head())
display(data_2.head())
display(data_3.head())

,product,predictions
51140,182.079984,195.101959
83095,144.009283,185.669565
51464,173.971897,185.331543
93073,162.810993,180.464229
52356,123.920559,177.592367


,product,predictions
26531,137.945408,140.360110
80439,137.945408,139.983277
55165,137.945408,139.901279
42738,137.945408,139.849860
38665,137.945408,139.815396


,product,predictions
3540,154.062998,176.976289
98714,98.664401,174.615753
25463,166.293929,174.546751
6647,152.294162,173.953743
69502,121.349438,173.949349


In [10]:
#функция для расчета прибыли
def profit_func(selected_data):
    #выбираем топ скважин
    selected = selected_data.head(best)
    #суммируем целевые значения объема сырья
    total_volume = selected['product'].sum()
    #рассчитываем прибыль
    profit = total_volume * (revenue_barrel * barrels_per_unit) - budget
    
    return profit

###### Выводы
- написали функцию для расчета прибыли 
- оставили в таблицах только предсказания и запасы сырья
- отсортировали по значениям, предсказанным моделью, выбрали топ 200 точек
- рассчитали прибыль для полученного объёма сырья

## Расчёт прибыли и рисков 

In [11]:
def bootstrap_profit(data, n_samples=1000, best=200):
    state = np.random.RandomState(12345)
    profits = []
    
    for i in range(n_samples):
        subsample_indices = state.choice(data.index, size=500, replace=True)
        subsample = data.loc[subsample_indices]
        subsample = subsample.sort_values(by='predictions', ascending=False).head(best)
        profit = profit_func(subsample)
        profits.append(profit)
    
    profits = pd.Series(profits)
    mean_profit = profits.mean()
    lower_quantile = profits.quantile(0.025)
    upper_quantile = profits.quantile(0.975)
    risk_of_loss = (profits < 0).mean()
    
    return mean_profit, lower_quantile, upper_quantile, risk_of_loss

profits_1 = bootstrap_profit(data_1)
profits_2 = bootstrap_profit(data_2)
profits_3 = bootstrap_profit(data_3)

def format_millions(number):
    return f"{round(number / 1_000_000)} млн. руб."

# Вывод результатов для региона 1
print(f"Регион 1: Средняя прибыль: {format_millions(profits_1[0])}, 95%-й доверительный интервал: от {format_millions(profits_1[1])} до {format_millions(profits_1[2])}, риск убытков: {profits_1[3]*100:.1f}%")

# Вывод результатов для региона 2
print(f"Регион 2: Средняя прибыль: {format_millions(profits_2[0])}, 95%-й доверительный интервал: от {format_millions(profits_2[1])} до {format_millions(profits_2[2])}, риск убытков: {profits_2[3]*100:.1f}%")

# Вывод результатов для региона 3
print(f"Регион 3: Средняя прибыль: {format_millions(profits_3[0])}, 95%-й доверительный интервал: от {format_millions(profits_3[1])} до {format_millions(profits_3[2])}, риск убытков: {profits_3[3]*100:.1f}%")

# Выбор региона для разработки скважин
if profits_1[3] < 0.025:
    print("Регион 1 подходит для разработки.")
if profits_2[3] < 0.025:
    print("Регион 2 подходит для разработки.")
if profits_3[3] < 0.025:
    print("Регион 3 подходит для разработки.")

Регион 1: Средняя прибыль: 441 млн. руб., 95%-й доверительный интервал: от -89 млн. руб. до 951 млн. руб., риск убытков: 5.6%
Регион 2: Средняя прибыль: 470 млн. руб., 95%-й доверительный интервал: от 72 млн. руб. до 894 млн. руб., риск убытков: 1.3%
Регион 3: Средняя прибыль: 375 млн. руб., 95%-й доверительный интервал: от -177 млн. руб. до 864 млн. руб., риск убытков: 8.1%
Регион 2 подходит для разработки.


## Общий вывод

На основании представленных данных можно сделать следующие выводы и рекомендации:

- Регион 1 имеет средний предсказанный запас сырья около 92.4 тыс. баррелей, с RMSE примерно 37.76.
- Регион 2 обладает меньшим средним предсказанным запасом сырья, около 68.7 тыс. баррелей, однако RMSE модели значительно ниже, всего 0.89, что указывает на высокую точность модели.
- Регион 3 имеет средний предсказанный запас сырья около 94.8 тыс. баррелей, с RMSE примерно 40.15.

###### Оценка прибыли:
- При оценке прибыли по регионам, наиболее перспективным оказывается регион 2. Средняя прибыль составляет примерно 469.7 миллионов, с хорошим 95%-м доверительным интервалом от 71.5 до 893.8 миллионов. Риск убытков составляет всего 1.3% (или 0.013).
- Регион 1 и регион 3 также имеют положительные средние прибыли, однако они сопряжены с более высоким риском убытков (5.6% для региона 1 и 8.1% для региона 3).

######  Рекомендации
Выбор региона 2 для разработки скважин представляется наиболее обоснованным с точки зрения минимизации рисков и максимизации прибыли. Этот регион демонстрирует наилучшие показатели средней прибыли, наименьший уровень риска убытков и высокую точность модели предсказания запасов сырья.